[View in Colaboratory](https://colab.research.google.com/github/Curiousss/InkerIntern/blob/master/Copy_of_DNST_CIFAR10_AUG.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 10
num_filter = 20
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
#num_filter = 12
#dropout_rate = 0.2
#l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 20)   540         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 20)   80          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 20)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 186s 4ms/step - loss: 1.5954 - acc: 0.4088 - val_loss: 1.5050 - val_acc: 0.4552
Epoch 2/50
19200/50000 [==========>...................] - ETA: 1:41 - loss: 1.2537 - acc: 0.5456

50000/50000 [==============================] - 174s 3ms/step - loss: 1.1820 - acc: 0.5720 - val_loss: 1.4878 - val_acc: 0.5322
Epoch 3/50
45568/50000 [==========================>...] - ETA: 14s - loss: 1.0362 - acc: 0.6290

50000/50000 [==============================] - 174s 3ms/step - loss: 1.0312 - acc: 0.6306 - val_loss: 1.4245 - val_acc: 0.5534
Epoch 4/50
50000/50000 [==============================] - 174s 3ms/step - loss: 0.9443 - acc: 0.6614 - val_loss: 1.4699 - val_acc: 0.5543
Epoch 5/50
 3328/50000 [>.............................] - ETA: 2:33 - loss: 0.9003 - acc: 0.6620

50000/50000 [==============================] - 174s 3ms/step - loss: 0.8792 - acc: 0.6856 - val_loss: 0.9197 - val_acc: 0.6816
Epoch 6/50
39552/50000 [======================>.......] - ETA: 34s - loss: 0.8275 - acc: 0.7045

50000/50000 [==============================] - 174s 3ms/step - loss: 0.8249 - acc: 0.7062 - val_loss: 0.8574 - val_acc: 0.7028
Epoch 7/50
50000/50000 [==============================] - 174s 3ms/step - loss: 0.7774 - acc: 0.7231 - val_loss: 1.1456 - val_acc: 0.6428
Epoch 8/50
 1920/50000 [>.............................] - ETA: 2:37 - loss: 0.7618 - acc: 0.7281

...

50000/50000 [==============================] - 175s 4ms/step - loss: 0.3402 - acc: 0.8801 - val_loss: 0.6745 - val_acc: 0.8151
Epoch 45/50
38656/50000 [======================>.......] - ETA: 37s - loss: 0.3283 - acc: 0.8844

50000/50000 [==============================] - 175s 4ms/step - loss: 0.3290 - acc: 0.8839 - val_loss: 0.6758 - val_acc: 0.8081
Epoch 46/50
50000/50000 [==============================] - 175s 4ms/step - loss: 0.3321 - acc: 0.8828 - val_loss: 0.7773 - val_acc: 0.7810
Epoch 47/50
 1536/50000 [..............................] - ETA: 2:40 - loss: 0.3322 - acc: 0.8835

50000/50000 [==============================] - 175s 4ms/step - loss: 0.3321 - acc: 0.8824 - val_loss: 0.8553 - val_acc: 0.7785
Epoch 48/50
38656/50000 [======================>.......] - ETA: 37s - loss: 0.3246 - acc: 0.8857

50000/50000 [==============================] - 175s 4ms/step - loss: 0.3277 - acc: 0.8841 - val_loss: 0.7030 - val_acc: 0.8140
Epoch 49/50
50000/50000 [==============================] - 175s 4ms/step - loss: 0.3259 - acc: 0.8848 - val_loss: 0.6331 - val_acc: 0.8226
Epoch 50/50
 1536/50000 [..............................] - ETA: 2:40 - loss: 0.2990 - acc: 0.8926

50000/50000 [==============================] - 175s 4ms/step - loss: 0.3158 - acc: 0.8871 - val_loss: 0.6154 - val_acc: 0.8298


In [13]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 15s 1ms/step
Test loss: 0.6154245536088944
Test accuracy: 0.8298


In [14]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')